In [ ]:
# a local RDF file? 
# Put in a local triple store and point your code to localhost

In [ ]:
# Use `rdflib` to query the rdflib.graph.Graph()

filename = "path/to/fileneme" #replace with something interesting
uri = "uri_of_interest" #replace with something interesting

import rdflib
import rdfextras
rdfextras.registerplugins() # so we can Graph.query()

g=rdflib.Graph()
g.parse(filename)
results = g.query("""
SELECT ?p ?o
WHERE {
<%s> ?p ?o.
}
ORDER BY (?p)
""" % uri) #get every predicate and object about the uri
